# Homework 3

Letizia Russo, Daniel Losada Molina and Michele Luca Puzzo

In [ ]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [3]:
import requests
import lxml
import shutil
#import functions as fun
import urllib.request
import urllib.error as uer
import time
import nltk
import datetime
import csv
import os

from tqdm import tqdm
from bs4 import BeautifulSoup
from pathlib import Path
from nltk.corpus import stopwords
from nltk.stem.snowball import PorterStemmer
from nltk.tokenize import RegexpTokenizer

In [12]:
nFolders = 299
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun','Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
url_file = 'urls.txt'
num_lines = sum(1 for line in open(url_file, encoding="utf-8"))
pagesPerFolder = num_lines/nFolders
outputTSV = "dataset.tsv"

In [5]:
    Path("directory").mkdir(exist_ok=True)
    for i in range(1,299+1):
        nomeCartella = 'cartelle/cartella{}'.format(i)
        Path(nomeCartella).mkdir(exist_ok=True)

In [87]:
def getUrl(lineNum, url_file):
    with open(url_file) as fp:
        for i, line in enumerate(fp):
            if i == lineNum-1:
                return line
            elif i >= lineNum:
                break

def downloadOneFile(pageNum):
    
    num_lines = sum(1 for line in open("urls.txt", encoding="utf-8"))
    pagesPerFolder = num_lines/nFolders
    url = getUrl(pageNum, "urls.txt")
    page = requests.get(url)
    soup = BeautifulSoup(page.content, features ="lxml")
    cartellaNumber = int(pageNum/(pagesPerFolder)) + 1
    print("Cartella" + str(cartellaNumber) + ".  Page_"+ str(pageNum))
    f = open("./cartelle/cartella{}".format(cartellaNumber) + "/page_{}.html".format(pageNum), "w",encoding="utf-8")
    f.write(soup.prettify())
    f.close()

In [89]:
downloadOneFile(1618)

Cartella26.  Page_1618


# 1. Data Collection

Since no dataset was provided for this homework, we had to create our own dataset by scraping the given website, which was done in the following steps.

## 1.1 Get the list of animes

Firstly, we had to get the list of all the anime we wanted to add to our dataset. This included all the anime found in the first 400 pages. Therefore, we created a function that scrapes a certain number of pages from an initial url, to extract the url of each of them. The initial url and the following number of pages are given as input, while the resulting urls are stored in a txt file called "urls.txt" .

In [4]:
def link_list(anime):
    with open('urls.txt', 'w') as file:
        for page in range(0, anime+50, 50):
            url = 'https://myanimelist.net/topanime.php?limit=' + str(page)
            response = requests.get(url)
            soup = BeautifulSoup(response.content)
            ranking_list = soup.find_all(class_='ranking-list')
            for l in ranking_list:
                link = l.find_all(class_='detail')[0].a['href']
                file.write("%s\n" % link)

In [ ]:
link_list(20000)     


100%|██████████| 20000/20000 [00:00<00:00, 950345.87it/s]


## 1.2 Crawl books

In [ ]:
def crawl(url_file):
    Path("directory").mkdir(exist_ok=True)
    for i in range(1,nFolders+1):
        nomeCartella = 'cartelle/cartella{}'.format(i)
        Path(nomeCartella).mkdir(exist_ok=True)
    num_lines = sum(1 for line in open(url_file, encoding="utf-8"))
    pagesPerFolder = num_lines/nFolders
    
    c = 0
    lastFolder = 1
    
    with open(url_file, "r", encoding="utf-8") as a:
        for i in range(c):
            print("skipping url: ", i)
            next(a)
        for i in a:
            page = requests.get(i)
            soup = BeautifulSoup(page.content, features ="lxml")
            number = int(c/(pagesPerFolder)) + 1
            if number != lastFolder:
                print("I'm waiting")
                lastFolder = number
                time.sleep(200)            
            print("Going to save in cartella" + str(number) + ", the page " + str(c+1))
            f = open("./cartelle/cartella{}".format(number) + "/page_{}.html".format(c+1), "w",encoding="utf-8")
            f.write(soup.prettify())
            f.close()

            file_size = os.path.getsize("./cartelle/cartella{}/page_{}.html".format(number, c + 1))
            if file_size < 50000:
                os.remove("./cartelle/cartella{}/page_{}.html".format(number, c + 1))
                page = requests.get(i)
                soup = BeautifulSoup(page.content, features ="lxml")
                f = open("./cartelle/cartella{}".format(number) + "/page_{}.html".format(c+1), "w",encoding="utf-8")
                f.write(soup.prettify())
                f.close()

            else:
                c = c + 1

In [19]:
crawl("urls.txt")

skipping url:  0
skipping url:  1
skipping url:  2
skipping url:  3
skipping url:  4
skipping url:  5
skipping url:  6
skipping url:  7
skipping url:  8
skipping url:  9
skipping url:  10
skipping url:  11
skipping url:  12
skipping url:  13
skipping url:  14
skipping url:  15
skipping url:  16
skipping url:  17
skipping url:  18
skipping url:  19
skipping url:  20
skipping url:  21
skipping url:  22
skipping url:  23
skipping url:  24
skipping url:  25
skipping url:  26
skipping url:  27
skipping url:  28
skipping url:  29
skipping url:  30
skipping url:  31
skipping url:  32
skipping url:  33
skipping url:  34
skipping url:  35
skipping url:  36
skipping url:  37
skipping url:  38
skipping url:  39
skipping url:  40
skipping url:  41
skipping url:  42
skipping url:  43
skipping url:  44
skipping url:  45
skipping url:  46
skipping url:  47
skipping url:  48
skipping url:  49
skipping url:  50
skipping url:  51
skipping url:  52
skipping url:  53
skipping url:  54
skipping url:  55
sk

skipping url:  634
skipping url:  635
skipping url:  636
skipping url:  637
skipping url:  638
skipping url:  639
skipping url:  640
skipping url:  641
skipping url:  642
skipping url:  643
skipping url:  644
skipping url:  645
skipping url:  646
skipping url:  647
skipping url:  648
skipping url:  649
skipping url:  650
skipping url:  651
skipping url:  652
skipping url:  653
skipping url:  654
skipping url:  655
skipping url:  656
skipping url:  657
skipping url:  658
skipping url:  659
skipping url:  660
skipping url:  661
skipping url:  662
skipping url:  663
skipping url:  664
skipping url:  665
skipping url:  666
skipping url:  667
skipping url:  668
skipping url:  669
skipping url:  670
skipping url:  671
skipping url:  672
skipping url:  673
skipping url:  674
skipping url:  675
skipping url:  676
skipping url:  677
skipping url:  678
skipping url:  679
skipping url:  680
skipping url:  681
skipping url:  682
skipping url:  683
skipping url:  684
skipping url:  685
skipping url

skipping url:  1133
skipping url:  1134
skipping url:  1135
skipping url:  1136
skipping url:  1137
skipping url:  1138
skipping url:  1139
skipping url:  1140
skipping url:  1141
skipping url:  1142
skipping url:  1143
skipping url:  1144
skipping url:  1145
skipping url:  1146
skipping url:  1147
skipping url:  1148
skipping url:  1149
skipping url:  1150
skipping url:  1151
skipping url:  1152
skipping url:  1153
skipping url:  1154
skipping url:  1155
skipping url:  1156
skipping url:  1157
skipping url:  1158
skipping url:  1159
skipping url:  1160
skipping url:  1161
skipping url:  1162
skipping url:  1163
skipping url:  1164
skipping url:  1165
skipping url:  1166
skipping url:  1167
skipping url:  1168
skipping url:  1169
skipping url:  1170
skipping url:  1171
skipping url:  1172
skipping url:  1173
skipping url:  1174
skipping url:  1175
skipping url:  1176
skipping url:  1177
skipping url:  1178
skipping url:  1179
skipping url:  1180
skipping url:  1181
skipping url:  1182


skipping url:  1883
skipping url:  1884
skipping url:  1885
skipping url:  1886
skipping url:  1887
skipping url:  1888
skipping url:  1889
skipping url:  1890
skipping url:  1891
skipping url:  1892
skipping url:  1893
skipping url:  1894
skipping url:  1895
skipping url:  1896
skipping url:  1897
skipping url:  1898
skipping url:  1899
skipping url:  1900
skipping url:  1901
skipping url:  1902
skipping url:  1903
skipping url:  1904
skipping url:  1905
skipping url:  1906
skipping url:  1907
skipping url:  1908
skipping url:  1909
skipping url:  1910
skipping url:  1911
skipping url:  1912
skipping url:  1913
skipping url:  1914
skipping url:  1915
skipping url:  1916
skipping url:  1917
skipping url:  1918
skipping url:  1919
skipping url:  1920
skipping url:  1921
skipping url:  1922
skipping url:  1923
skipping url:  1924
skipping url:  1925
skipping url:  1926
skipping url:  1927
skipping url:  1928
skipping url:  1929
skipping url:  1930
skipping url:  1931
skipping url:  1932


skipping url:  2633
skipping url:  2634
skipping url:  2635
skipping url:  2636
skipping url:  2637
skipping url:  2638
skipping url:  2639
skipping url:  2640
skipping url:  2641
skipping url:  2642
skipping url:  2643
skipping url:  2644
skipping url:  2645
skipping url:  2646
skipping url:  2647
skipping url:  2648
skipping url:  2649
skipping url:  2650
skipping url:  2651
skipping url:  2652
skipping url:  2653
skipping url:  2654
skipping url:  2655
skipping url:  2656
skipping url:  2657
skipping url:  2658
skipping url:  2659
skipping url:  2660
skipping url:  2661
skipping url:  2662
skipping url:  2663
skipping url:  2664
skipping url:  2665
skipping url:  2666
skipping url:  2667
skipping url:  2668
skipping url:  2669
skipping url:  2670
skipping url:  2671
skipping url:  2672
skipping url:  2673
skipping url:  2674
skipping url:  2675
skipping url:  2676
skipping url:  2677
skipping url:  2678
skipping url:  2679
skipping url:  2680
skipping url:  2681
skipping url:  2682


...

Going to save in cartella135, the page 8620
Going to save in cartella135, the page 8621
Going to save in cartella135, the page 8622
Going to save in cartella135, the page 8623
Going to save in cartella135, the page 8624
Going to save in cartella135, the page 8625
Going to save in cartella135, the page 8626
Going to save in cartella135, the page 8627
Going to save in cartella135, the page 8628
Going to save in cartella135, the page 8629
Going to save in cartella135, the page 8630
Going to save in cartella135, the page 8631
Going to save in cartella135, the page 8632
Going to save in cartella135, the page 8633
Going to save in cartella135, the page 8634
Going to save in cartella135, the page 8635
Going to save in cartella135, the page 8636
Going to save in cartella135, the page 8637
Going to save in cartella135, the page 8638
Going to save in cartella135, the page 8639
Going to save in cartella135, the page 8640
I'm waiting
Going to save in cartella136, the page 8641
Going to save in car

Going to save in cartella138, the page 8806
Going to save in cartella138, the page 8807
Going to save in cartella138, the page 8808
Going to save in cartella138, the page 8809
Going to save in cartella138, the page 8810
Going to save in cartella138, the page 8811
Going to save in cartella138, the page 8812
Going to save in cartella138, the page 8813
Going to save in cartella138, the page 8814
Going to save in cartella138, the page 8815
Going to save in cartella138, the page 8816
Going to save in cartella138, the page 8817
Going to save in cartella138, the page 8818
Going to save in cartella138, the page 8819
Going to save in cartella138, the page 8820
Going to save in cartella138, the page 8821
Going to save in cartella138, the page 8822
Going to save in cartella138, the page 8823
Going to save in cartella138, the page 8824
Going to save in cartella138, the page 8825
Going to save in cartella138, the page 8826
Going to save in cartella138, the page 8827
Going to save in cartella138, th

Going to save in cartella141, the page 8992
Going to save in cartella141, the page 8993
Going to save in cartella141, the page 8994
Going to save in cartella141, the page 8995
Going to save in cartella141, the page 8996
Going to save in cartella141, the page 8997
Going to save in cartella141, the page 8998
Going to save in cartella141, the page 8999
Going to save in cartella141, the page 9000
Going to save in cartella141, the page 9001
Going to save in cartella141, the page 9002
Going to save in cartella141, the page 9003
Going to save in cartella141, the page 9004
Going to save in cartella141, the page 9005
Going to save in cartella141, the page 9006
Going to save in cartella141, the page 9007
Going to save in cartella141, the page 9008
Going to save in cartella141, the page 9009
Going to save in cartella141, the page 9010
Going to save in cartella141, the page 9011
Going to save in cartella141, the page 9012
Going to save in cartella141, the page 9013
Going to save in cartella141, th

Going to save in cartella144, the page 9178
Going to save in cartella144, the page 9179
Going to save in cartella144, the page 9180
Going to save in cartella144, the page 9181
Going to save in cartella144, the page 9182
Going to save in cartella144, the page 9183
Going to save in cartella144, the page 9184
Going to save in cartella144, the page 9185
Going to save in cartella144, the page 9186
Going to save in cartella144, the page 9187
Going to save in cartella144, the page 9188
Going to save in cartella144, the page 9189
Going to save in cartella144, the page 9190
Going to save in cartella144, the page 9191
Going to save in cartella144, the page 9192
Going to save in cartella144, the page 9193
Going to save in cartella144, the page 9194
Going to save in cartella144, the page 9195
Going to save in cartella144, the page 9196
Going to save in cartella144, the page 9197
Going to save in cartella144, the page 9198
Going to save in cartella144, the page 9199
Going to save in cartella144, th

KeyboardInterrupt: 

## 1.3 Parse downloaded pages

In [80]:
def findField(array, word):
    for x in array:
        #print("#####################################")
        #print(" ".join((x.find("span").text).split()))
        #print("#####################################")
        if " ".join((x.find("span").text).split()) == word:
            return x
    return -1       


def extractData(pagePath):
    with open(pagePath, encoding="utf-8") as fp:
        soup = BeautifulSoup(fp, "html.parser")
        animeTitle = " ".join((soup.html.head.title.text).split())
        
        #print("animeTitle: ", animeTitle)
        animeInfo = (soup.find_all("div",  attrs={ "class" : "spaceit_pad"}))
        #print("Animeinfo: ", animeInfo)
        
        aux = findField(animeInfo, "Type:")
        animeType = ""
        if aux.a != None:
            animeType = " ".join((aux.a.text).split())
        else:
            animeType = " ".join((findField(animeInfo, "Type:").findAll(text=True, recursive=False))).split()[0]
        #print("animeType :", animeType)
        
        numEpisodes = list(filter(lambda y: y != '',list(map(lambda x: " ".join((x).split()),findField(animeInfo, "Episodes:").findAll(text=True, recursive=False)) )))[0]
        animeNumEpisode = ""
        if numEpisodes.isdigit():
            animeNumEpisode = int(numEpisodes)
        #print("animeNumEpisode: ", animeNumEpisode)
        
        aired = list(filter(lambda y: y != '',list(map(lambda x: " ".join((x).split()),findField(animeInfo, "Aired:").findAll(text=True, recursive=False)) )))[0].split('to')
        if len(aired[0].split()) == 3:
            releaseDate = datetime.datetime(int(aired[0].split()[2]), months.index(aired[0].split()[0])+1 , int(aired[0].split()[1][:1]))
        else:
            releaseDate = ""
        if len(aired) == 2 and len(aired[1].split()) == 3:
            endDate = datetime.datetime(int(aired[1].split()[2]), months.index(aired[1].split()[0])+1 , int(aired[1].split()[1][:1]))
        else:
            endDate = ""
        #print("releaseDate: ", releaseDate)
        #print("endDate: ", endDate)
        
        animeNumMembers = int(list(filter(lambda y: y != '',list(map(lambda x: " ".join((x).split()),findField(animeInfo, "Members:").findAll(text=True, recursive=False)) )))[0].replace(",", ""))
        #print("animeNumMembers: ", animeNumMembers)
        
        animeScore = float(" ".join((soup.find_all("span", itemprop = "ratingValue")[0].text).split()))
        #print("animeScore: ", animeScore)
        
        animeUsers = int(" ".join((soup.find_all("span", itemprop = "ratingCount")[0].text).split()))
        #print("animeUsers: ", animeUsers)
        
        animeRank = int(list(filter(lambda x: x[0] == '#',findField(animeInfo, "Ranked:").text.split()))[0].replace('#',''))
        #print("animeRank: ", animeRank)

        animePopularity = int(list(filter(lambda y: y != '',list(map(lambda x: " ".join((x).split()),findField(animeInfo, "Popularity:").findAll(text=True, recursive=False)) )))[0].replace("#", ""))
        #print("animePopularity: ", animePopularity)
        
        animeDescription = soup.find_all("p", itemprop = "description")[0].text.strip().replace('\n', '').replace('  ', '')
        #print("animeDescription: ", animeDescription)
        
        animeRelatedAHref = []
        if soup.find(name="table",attrs={"class":"anime_detail_related_anime"}) != None:
            animeRelatedAHref = soup.find(name="table",attrs={"class":"anime_detail_related_anime"}).findChildren('a', href=True)
        animeRelated = []
        for x in animeRelatedAHref:
            aux = " ".join((x.text).split())
            if aux not in animeRelated:
                animeRelated.append(aux)
        #print("animeRelated: ", animeRelated)
        
        animeCharactersRaw = soup.find_all("h3", attrs={"class" : "h3_characters_voice_actors"})
        animeCharacters = []
        for x in animeCharactersRaw:
            animeCharacters.append(" ".join((x.text).split()))
        #print("animeCharacters: ", animeCharacters)
        
        animeVoicesRaw = soup.find_all("td", attrs={"class" : "va-t ar pl4 pr4"})
        animeVoices = []
        for x in animeVoicesRaw:
            animeVoices.append(" ".join((x.contents[1].text).split()))
        #print("animeVoices: ", animeVoices)
        
        animeStaff = ""
        if(len(soup.find_all("div", attrs={"class" : "detail-characters-list clearfix"})) == 2):
            aux = soup.find_all("div", attrs={"class" : "detail-characters-list clearfix"})[1]
            animeStaffRaw = []
            for x in aux.findChildren('a'):
                if (" ".join((x.text).split()) != ''):
                    animeStaffRaw.append(" ".join((x.text).split()))
            animeStaffTaskRaw = []
            for x in aux.findChildren('small'):
                animeStaffTaskRaw.append(" ".join((x.text).split()))
            #print("animeStaffRaw: ", animeStaffRaw)
            #print("animeStaffTaskRaw: ", animeStaffTaskRaw)
            animeStaff = [list(a) for a in zip(animeStaffRaw, animeStaffTaskRaw)]

        
        #print("animeStaff: ", animeStaff)
        
        return [animeTitle, animeType, animeNumEpisode, releaseDate, endDate, animeNumMembers, animeScore, animeUsers, animeRank, animePopularity, animeDescription, animeRelated, animeCharacters, animeVoices, animeStaff]
    

def tsvGenerator():
    c = 579
    with open(outputTSV, 'wt', encoding="utf-8") as out_file:
        tsv_writer = csv.writer(out_file, delimiter='\t')
        tsv_writer.writerow(["animeTitle", "animeType", "animeNumEpisode", "releaseDate", "endDate", "animeNumMembers", "animeScore", "animeUsers", "animeRank", "animePopularity", "animeDescription", "animeRelated", "animeCharacters", "animeVoices", "animeStaff"])
        for x in range(1, num_lines +1):

            number = int(c/(pagesPerFolder)) + 1
            path = "./cartelle/cartella{}".format(number) + "/page_{}.html".format(c+1)
            print("path: ", path)
            tsv_writer.writerow(extractData(path))
            c = c + 1


In [81]:
tsvGenerator()

path:  ./cartelle/cartella10/page_580.html
path:  ./cartelle/cartella10/page_581.html
path:  ./cartelle/cartella10/page_582.html
path:  ./cartelle/cartella10/page_583.html
path:  ./cartelle/cartella10/page_584.html
path:  ./cartelle/cartella10/page_585.html
path:  ./cartelle/cartella10/page_586.html
path:  ./cartelle/cartella10/page_587.html
path:  ./cartelle/cartella10/page_588.html
path:  ./cartelle/cartella10/page_589.html
path:  ./cartelle/cartella10/page_590.html
path:  ./cartelle/cartella10/page_591.html
path:  ./cartelle/cartella10/page_592.html
path:  ./cartelle/cartella10/page_593.html
path:  ./cartelle/cartella10/page_594.html
path:  ./cartelle/cartella10/page_595.html
path:  ./cartelle/cartella10/page_596.html
path:  ./cartelle/cartella10/page_597.html
path:  ./cartelle/cartella10/page_598.html
path:  ./cartelle/cartella10/page_599.html
path:  ./cartelle/cartella10/page_600.html
path:  ./cartelle/cartella10/page_601.html
path:  ./cartelle/cartella10/page_602.html
path:  ./ca

path:  ./cartelle/cartella13/page_772.html
path:  ./cartelle/cartella13/page_773.html
path:  ./cartelle/cartella13/page_774.html
path:  ./cartelle/cartella13/page_775.html
path:  ./cartelle/cartella13/page_776.html
path:  ./cartelle/cartella13/page_777.html
path:  ./cartelle/cartella13/page_778.html
path:  ./cartelle/cartella13/page_779.html
path:  ./cartelle/cartella13/page_780.html
path:  ./cartelle/cartella13/page_781.html
path:  ./cartelle/cartella13/page_782.html
path:  ./cartelle/cartella13/page_783.html
path:  ./cartelle/cartella13/page_784.html
path:  ./cartelle/cartella13/page_785.html
path:  ./cartelle/cartella13/page_786.html
path:  ./cartelle/cartella13/page_787.html
path:  ./cartelle/cartella13/page_788.html
path:  ./cartelle/cartella13/page_789.html
path:  ./cartelle/cartella13/page_790.html
path:  ./cartelle/cartella13/page_791.html
path:  ./cartelle/cartella13/page_792.html
path:  ./cartelle/cartella13/page_793.html
path:  ./cartelle/cartella13/page_794.html
path:  ./ca

path:  ./cartelle/cartella16/page_964.html
path:  ./cartelle/cartella16/page_965.html
path:  ./cartelle/cartella16/page_966.html
path:  ./cartelle/cartella16/page_967.html
path:  ./cartelle/cartella16/page_968.html
path:  ./cartelle/cartella16/page_969.html
path:  ./cartelle/cartella16/page_970.html
path:  ./cartelle/cartella16/page_971.html
path:  ./cartelle/cartella16/page_972.html
path:  ./cartelle/cartella16/page_973.html
path:  ./cartelle/cartella16/page_974.html
path:  ./cartelle/cartella16/page_975.html
path:  ./cartelle/cartella16/page_976.html
path:  ./cartelle/cartella16/page_977.html
path:  ./cartelle/cartella16/page_978.html
path:  ./cartelle/cartella16/page_979.html
path:  ./cartelle/cartella16/page_980.html
path:  ./cartelle/cartella16/page_981.html
path:  ./cartelle/cartella16/page_982.html
path:  ./cartelle/cartella16/page_983.html
path:  ./cartelle/cartella16/page_984.html
path:  ./cartelle/cartella16/page_985.html
path:  ./cartelle/cartella16/page_986.html
path:  ./ca

path:  ./cartelle/cartella19/page_1153.html
path:  ./cartelle/cartella19/page_1154.html
path:  ./cartelle/cartella19/page_1155.html
path:  ./cartelle/cartella19/page_1156.html
path:  ./cartelle/cartella19/page_1157.html
path:  ./cartelle/cartella19/page_1158.html
path:  ./cartelle/cartella19/page_1159.html
path:  ./cartelle/cartella19/page_1160.html
path:  ./cartelle/cartella19/page_1161.html
path:  ./cartelle/cartella19/page_1162.html
path:  ./cartelle/cartella19/page_1163.html
path:  ./cartelle/cartella19/page_1164.html
path:  ./cartelle/cartella19/page_1165.html
path:  ./cartelle/cartella19/page_1166.html
path:  ./cartelle/cartella19/page_1167.html
path:  ./cartelle/cartella19/page_1168.html
path:  ./cartelle/cartella19/page_1169.html
path:  ./cartelle/cartella19/page_1170.html
path:  ./cartelle/cartella19/page_1171.html
path:  ./cartelle/cartella19/page_1172.html
path:  ./cartelle/cartella19/page_1173.html
path:  ./cartelle/cartella19/page_1174.html
path:  ./cartelle/cartella19/pag

path:  ./cartelle/cartella21/page_1341.html
path:  ./cartelle/cartella21/page_1342.html
path:  ./cartelle/cartella21/page_1343.html
path:  ./cartelle/cartella21/page_1344.html
path:  ./cartelle/cartella22/page_1345.html
path:  ./cartelle/cartella22/page_1346.html
path:  ./cartelle/cartella22/page_1347.html
path:  ./cartelle/cartella22/page_1348.html
path:  ./cartelle/cartella22/page_1349.html
path:  ./cartelle/cartella22/page_1350.html
path:  ./cartelle/cartella22/page_1351.html
path:  ./cartelle/cartella22/page_1352.html
path:  ./cartelle/cartella22/page_1353.html
path:  ./cartelle/cartella22/page_1354.html
path:  ./cartelle/cartella22/page_1355.html
path:  ./cartelle/cartella22/page_1356.html
path:  ./cartelle/cartella22/page_1357.html
path:  ./cartelle/cartella22/page_1358.html
path:  ./cartelle/cartella22/page_1359.html
path:  ./cartelle/cartella22/page_1360.html
path:  ./cartelle/cartella22/page_1361.html
path:  ./cartelle/cartella22/page_1362.html
path:  ./cartelle/cartella22/pag

path:  ./cartelle/cartella24/page_1528.html
path:  ./cartelle/cartella24/page_1529.html
path:  ./cartelle/cartella24/page_1530.html
path:  ./cartelle/cartella24/page_1531.html
path:  ./cartelle/cartella24/page_1532.html
path:  ./cartelle/cartella24/page_1533.html
path:  ./cartelle/cartella24/page_1534.html
path:  ./cartelle/cartella24/page_1535.html
path:  ./cartelle/cartella24/page_1536.html
path:  ./cartelle/cartella25/page_1537.html
path:  ./cartelle/cartella25/page_1538.html
path:  ./cartelle/cartella25/page_1539.html
path:  ./cartelle/cartella25/page_1540.html
path:  ./cartelle/cartella25/page_1541.html
path:  ./cartelle/cartella25/page_1542.html
path:  ./cartelle/cartella25/page_1543.html
path:  ./cartelle/cartella25/page_1544.html
path:  ./cartelle/cartella25/page_1545.html
path:  ./cartelle/cartella25/page_1546.html
path:  ./cartelle/cartella25/page_1547.html
path:  ./cartelle/cartella25/page_1548.html
path:  ./cartelle/cartella25/page_1549.html
path:  ./cartelle/cartella25/pag

FileNotFoundError: [Errno 2] No such file or directory: './cartelle/cartella26/page_1618.html'

# 2. Search Engine

In [ ]:
def clean_text(text):
    stop_words = set(stopwords.words('english')) # obtain the stop words
    good_words = [] # save the correct words to consider like tokens
    tokenizer = RegexpTokenizer("[\w']+") # function to recognize the tokens
    words = tokenizer.tokenize(text) # tokenize the text 
    for word in words:
        # check if the word is lower and it isn't a stop word or a number
        if word.lower() not in stop_words and word.isalpha(): 
            word = PorterStemmer().stem(word) # use the stemmer function
            good_words.append(word.lower()) # insert the good token to lower case
        
    return good_words

In [ ]:
a = "I'm Michele, and I live in Ostia"

In [ ]:
clean_text(a)

['michel', 'live', 'ostia']

In [ ]:
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


In [ ]:
def cleaner(text):
    stop_words = set(stopwords.words('english')) # retrieve stop words
    tokenizer = RegexpTokenizer("[\w']+") # recognize the tokens
    parole = tokenizer.tokenize(text) # tokenize the text
    correct_words = [] # save the correct words to consider like tokens
    for word in parole:
        # check if the word is lower and it isn't a stop word or a number
        if word.lower() not in stop_words and word.isalpha(): 
            word = PorterStemmer().stem(word) # use the stemmer function
            correct_words.append(word.lower()) # insert the good token to lower case
        
    return correct_words

def create_vocabulary(data):
    ### Input == I use like input the dataset obtain in exercise 1 where i apply the clean text function
    ### Output == I obtain a vocabulary, the keys are all tokens (with no repeat) contained in the synopsis for the each rows
    ### for each token I define the index of the rows where the token is in the synopsis
    voc = {}
    for i, row in data.iterrows():
            if len(data.at[i, "Synopsis"]) > 0:  # check if the list is empty or not to avoid the eventually error
                for word in data.at[i, "Synopsis"]: # bring the token from the list
                    if word in voc.keys(): # insert the token into the vocabulary with the documents where this is present
                        if i not in voc[word]:
                            voc[word].append(i)
                    else:
                        voc[word] = [i]
    return voc


In [ ]:
# create the inverted list according to the professors' requests
def create_inverted_list(vocabulary):
    inv_lst = {}
    indexes = list(vocabulary.keys()) # return the indexes list of the vocabulary
    for key in vocabulary.keys():
        term_id = indexes.index(key) # find the corresponding id from the vocabulary
        inv_lst[term_id] = vocabulary[key] # insert the list of documents into the inverted list
    
    return inv_lst

# map the interested word with corresponding term_id_i
def map_terms_id(vocabulary, cleanQString):
    # find each term_id
    term_id = []  # this is another function useful for mapping the term_id_i with the word into the vocabulary
    indexes = list(vocabulary.keys()) # return the indexes list of the vocabulary
    for token in cleanQString:
        term = indexes.index(token)
        term_id.append(term) # append the id that we want to make the score
        
    return term_id

# clean user's query
def clean_query(query):
    clean_string = query.split(" ")
    
    stop_words = set(stopwords.words('english')) # obtain the stop words
    good_words = [] # save the correct words to consider like tokens
    tokenizer = RegexpTokenizer("[\w']+") # function to recognize the tokens
    
    for item in clean_string:
        item = tokenizer.tokenize(word) # tokenize the text
        for w in item:
            if w.lower() not in stop_words and w.isalpha(): 
                w = PorterStemmer().stem(w) # use the stemmer function
                good_words.append(w.lower()) # insert the good token to lower case
    
    return good_words

# the search engine 1
def Search1(cleanQString, vocab, df, inv_lst):
    term_id = map_terms_id(vocab, cleanQString) # return the corresponding id of those terms

    # find the common documents where those terms are present
    intersection_list = []
    for term in term_id:
        if not intersection_list:
            intersection_list = inv_lst[term] # if the intersection list is empty insert the first list of the first token
        else:
            intersection_list = set(intersection_list).intersection(set(inv_lst[term])) # make the intersection, this respect the properties of the sets

    new_df = pd.DataFrame(columns=['animeTitle', 'animeDescription', 'Url']) # create the new dataset according to the professors' requests
    for row in intersection_list:
        #append row to the dataframe
        new_row = {'animeTitle': df.loc[row, "animeTitle"], 'animeDescription': df.loc[row, "animeDescription"], 'Url': df.loc[row, "Url"]}
        new_df = new_df.append(new_row, ignore_index=True)
        
    return new_df




In [ ]:
df = openCSV("finaloutput.csv")

In [ ]:
vocabulary = create_vocabulary(df1)

In [ ]:
a_file = open("vocabulary.json", "w")
a_file = json.dump(vocabulary, a_file)

In [ ]:
a_file = open("vocabulary.json", "r")
vocabulary = json.load(a_file)

In [ ]:
inv_lst = create_inverted_list(vocabulary)

In [ ]:
cleanQString = cleanQuery("sayan race")

In [ ]:
new_dataset = search_engine1(cleanQString, vocab, df, inv_lst).head(10)

# 5. Algorithmic question

In [1]:
def optimalSol(arr):
    incl = arr[0]
    excl = 0
    indexOfOptimalSol = [0]
    #print("$$$$$$$$$$$$$$$$$$$$")
    #print("index: ", 0)
    #print("incl = ", incl)
    #print("excl = ", excl)
    #print("indexOfOptimalSol: ", indexOfOptimalSol)
    #print("$$$$$$$$$$$$$$$$$$$$")
    lastElem = 0
    lastIndexArrayState = []
    for i in range(1,len(arr)):
        oldIncl = incl
        if excl+arr[i] > incl:
            aux = indexOfOptimalSol
            if i - 1 in indexOfOptimalSol:
                indexOfOptimalSol = lastIndexArrayState + [i]
            else: 
                indexOfOptimalSol = indexOfOptimalSol + [i]
            lastIndexArrayState = aux
            incl = excl+arr[i]
        excl = oldIncl
        #print("$$$$$$$$$$$$$$$$$$$$")
        #print("index: ", i)
        #print("incl = ", incl)
        #print("excl = ", excl)
        #print("indexOfOptimalSol: ", indexOfOptimalSol)
        #print("$$$$$$$$$$$$$$$$$$$$")
        lastElem = i
        
        
    
    sol = list(map(lambda x: arr[x],indexOfOptimalSol))
    print("SOLUTION: ", incl)
    print("The solution is the sum of the elements: ", sol)
    return incl, sol
    

In [2]:
optimalSol([30, 40, 25, 50, 30, 20])

SOLUTION:  110
The solution is the sum of the elements:  [40, 50, 20]


(110, [40, 50, 20])